## Example of displaying heatmap using folium (English)  

### Import libraries  

In [1]:
import pandas as pd
import folium

### Parameters

In [2]:
csv_in = 'pop_move_H27_H28_en-utf8.csv'

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

### Read CSV file

In [3]:
df = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
print(df.shape)
print(df.info())
display(df.head())
#display(df)

(47, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Pref      47 non-null     object
 1   Move_in   47 non-null     int64 
 2   Move_out  47 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB
None


,Pref,Move_in,Move_out
0,Hokkaido,48544,55418
1,Aomori,17061,23384
2,Iwate,17113,20983
3,Miyagi,47124,47607
4,Akita,11353,15751


### Calculation of increase/decrease of population  

In [4]:
df['diff'] = df['Move_in'] - df['Move_out']
display(df.head())

,Pref,Move_in,Move_out,diff
0,Hokkaido,48544,55418,-6874
1,Aomori,17061,23384,-6323
2,Iwate,17113,20983,-3870
3,Miyagi,47124,47607,-483
4,Akita,11353,15751,-4398


### Check data  

In [5]:
print(df.loc[df['diff'].idxmin()])
print(df.loc[df['diff'].idxmax()])

Pref        Hokkaido
Move_in        48544
Move_out       55418
diff           -6874
Name: 0, dtype: object
Pref         Tokyo
Move_in     413444
Move_out    339267
diff         74177
Name: 12, dtype: object


### Parameters2 (for lat/lon)   

In [6]:
csv_in2 = 'h3010pref_pos_utf8_en.csv'

### Read CSV file

In [7]:
df_pos = pd.read_csv(csv_in2, delimiter=',', skiprows=0, header=0)
print(df_pos.shape)
display(df_pos.head())

(47, 3)


,Pref,lat,lon
0,Hokkaido,43.064278,141.346785
1,Aomori,40.824338,140.740087
2,Iwate,39.703647,141.152592
3,Miyagi,38.268803,140.871846
4,Akita,39.718058,140.103250


### Merge DataFrames to add lat/lon to pref data  

In [8]:
df_merged = pd.merge(df, df_pos, on='Pref', how='left')
print(df_merged.shape)
display(df_merged.head())

(47, 6)


,Pref,Move_in,Move_out,diff,lat,lon
0,Hokkaido,48544,55418,-6874,43.064278,141.346785
1,Aomori,17061,23384,-6323,40.824338,140.740087
2,Iwate,17113,20983,-3870,39.703647,141.152592
3,Miyagi,47124,47607,-483,38.268803,140.871846
4,Akita,11353,15751,-4398,39.718058,140.103250


### Make data for heatmap  
[ [lat2, lon2, value2], [lat2, lon2, value2], ... ]  

In [9]:
df_heat = df_merged[['lat', 'lon', 'diff']]
print(df_heat.shape)
display(df_heat.head())

(47, 3)


,lat,lon,diff
0,43.064278,141.346785,-6874
1,40.824338,140.740087,-6323
2,39.703647,141.152592,-3870
3,38.268803,140.871846,-483
4,39.718058,140.103250,-4398


In [10]:
ar_heat = []
for i,r in df_heat.iterrows():
    ar_heat.append(r)
print(ar_heat[:3])

[lat       43.064278
lon      141.346785
diff   -6874.000000
Name: 0, dtype: float64, lat       40.824338
lon      140.740087
diff   -6323.000000
Name: 1, dtype: float64, lat       39.703647
lon      141.152592
diff   -3870.000000
Name: 2, dtype: float64]


### Make map  

In [11]:
from folium.plugins import HeatMap

m = folium.Map(location=[35, 140], zoom_start=4)
m.add_child(HeatMap(ar_heat, radius=12))
display(m)

### Add color bar  

In [12]:
from folium.plugins import HeatMap
import branca.colormap as cm

m = folium.Map(location=[35, 140], zoom_start=4)
m.add_child(HeatMap(ar_heat, radius=12,
                    gradient={0:'blue', 0.5:'lime', 1:'red'}))

min = df_heat['diff'].min()
max = df_heat['diff'].max()
colors = ['blue', 'lime', 'red']
colormap = cm.LinearColormap(colors, vmin=min, vmax=max).to_step(10000)
colormap.caption = 'Increase/Decrease of Population'
m.add_child(colormap)

display(m)